# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
tf.random.set_seed(42)

# Fonctions utiles pour récupérer la data.

In [ ]:
def load_data(n):
    data = pd.read_csv('../data/train.new.csv')
    return data[0:n]

def select_variables(data):
    data.dropna(axis=0, inplace=True)
    y = data['smoking'] # récupérer la colonne survived et la mettre dans y
    X = data.drop('country', axis=1)
    X = X.drop('id', axis=1)
    X = X.replace(["M", "F"], [0, 1])
    X = X.drop('smoking', axis=1)
    return X,y


# Exemple de code pour récupérer la data et la préparer.

In [ ]:
normalizer = RobustScaler() # normalise les données
data=load_data(-1)
# sélectionner les variables
X,y = select_variables(data)
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=0, test_size=0.2)
X_train = normalizer.fit_transform(X_train)
X_test = normalizer.transform(X_test)

# Création du modèle

In [ ]:
from tensorflow.keras import layers
from kerastuner.tuners import Hyperband

def build_model(hp):
    model = keras.Sequential()

    # Add layers based on the best hyperparameters
    for _ in range(hp.Int('nb_layers', min_value=2, max_value=5)):
        if model.layers != []:
            if hp.Choice('add_dropout_and_batch', values=[True, False]):
                model.add(layers.Dropout(hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))
                model.add(layers.BatchNormalization())
        model.add(layers.Dense(units=hp.Int('nb_units', min_value=32, max_value=528, step=64),
                               activation='relu'))
        model.add(layers.C)

    model.add(layers.Dense(1, activation='sigmoid'))
    # Decay learning rate
    learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]),
        decay_steps=10000,
        decay_rate=0.9)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

def build_hierarchical_model(hp):
    model = keras.Sequential()

    # Add layers based on the best hyperparameters
    tot_units = hp.Int('nb_units', min_value=32, max_value=528, step=64)
    #each layer have 2 times less units than the previous one
    while tot_units >= 32:
        if (model.layers != []):
            model.add(layers.Dropout(hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))
            model.add(layers.BatchNormalization())
        model.add(layers.Dense(units=tot_units, activation='relu'))
        tot_units = tot_units // 2
    model.add(layers.Dense(1, activation='sigmoid'))
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

def tune_model(X_train, y_train, X_test, y_test):
    tuner = Hyperband(build_hierarchical_model,
                      objective='val_accuracy',
                      max_epochs=30,
                      factor=3,
                      hyperband_iterations=8,
                      directory='my_dir',
                      project_name='intro_to_kt')

    tuner.search(X_train, y_train,
                 epochs=50,
                 validation_data=(X_test, y_test))

    best_hps = tuner.get_best_hyperparameters()[0]

    return best_hps

Récupération des meilleurs hyperparamètres

In [ ]:
best_hps = tune_model(X_train, y_train, X_test, y_test)
best_hps.get('learning_rate'), best_hps.get('nb_units'), best_hps.get('dropout')

# # Meilleur modèle pour l'instant :
- 3 couches cachées
- Une de 96 neurones
- Une de 48 neurones
- Une de 24 neurones
- Dropout de 0.2
- Batch Normalisation avant l'activation
- Optimiseur : Adam
- Learning rate : 0.001 (test en cours avec un learning rate dynamique)

# Résultats :
- 0.777 de précision sur le test set

# Créer son modèle défini si demandé dans le sujet
# Layers input -> X_train.shape[1] (nombre de colonnes)
# Dense -> nombre de neurones
# Dropout -> pourcentage de neurones à désactiver pour éviter l'overfitting
# BatchNormalization -> normalise les données pour éviter l'overfitting
# Dense -> 1 neurone pour la sortie
# Optimizer -> Adam (le plus utilisé)
# lr -> learning rate (taux d'apprentissage)
# loss -> binary_crossentropy (car problème de classification binaire)

In [ ]:
model = keras.Sequential()
model.add(layers.Input(shape=(X_train.shape[1],)))
model.add(layers.Dense(units=526, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.BatchNormalization())
model.add(layers.Dense(units=128, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.BatchNormalization())
model.add(layers.Dense(units=32, activation='relu'))
model.add(layers.Dropout(best_hps.get('dropout')))
model.add(layers.BatchNormalization())
model.add(layers.Dense(1, activation='sigmoid'))
lr = best_hps.get('learning_rate')
model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
              loss='binary_crossentropy',
              metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), batch_size=16)

# Affichage des résultats

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.legend(['Train', 'Test'], loc='right')
plt.title('Model accuracy with best hyperparameters')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')

# Matrice de confusion

In [ ]:
def visualiser_confusion(model, X_test, y_test):
    y_pred = model.predict(X_test)
    # Get class labels
    y_classes = np.argmax(y_pred, axis=-1)

    cm = confusion_matrix(y_test, y_classes)
    #disp= ConfusionMatrixDisplay(confusion_matrix=cm)

    sns.heatmap(cm, annot=True, annot_kws={"size": 12}) # font size
    plt.show()
visualiser_confusion(model, X_test, y_test)

# Evaluation du modèle

In [ ]:
model.evaluate(X_test, y_test)

# Prédiction sur le test set

In [ ]:
test = pd.read_csv('../data/test.new.csv')
test = test.drop('country', axis=1)
test = test.drop('id', axis=1)
test = test.replace(["M", "F"], [0, 1])
test = normalizer.transform(test)
y_pred = model.predict(test)
y_pred = np.argmax(y_pred, axis=-1)
y_pred = pd.DataFrame(y_pred)
y_pred.to_csv('../data/y_pred.csv', index=False)

# Sauvegarde du modèle

In [ ]:
model.save('../models/model.h5')

# Chargement du modèle

In [ ]:
model = keras.models.load_model('../models/model.h5')

# Sauvegarde de l'historique

In [ ]:
hist_df = pd.DataFrame(history.history)
hist_csv_file = '../models/history.csv'
hist_df.to_csv(hist_csv_file, index=False)

# Fonctions d'activation pour la classification
- sigmoid : 0 ou 1 pour la classification binaire
- softmax : probabilité pour chaque classe pour la classification multiclasse
- relu : 0 ou 1 pour la classification binaire
- mish : -1 ou 1 pour la classification binaire couche cachée
- exponential : 0 à +inf pour la régression
- linear : -inf à +inf pour la régression

# Préprocessing à utiliser sur les données
- Gestion des collonnes textuelles -> LabelEncoder (pour les variables catégorielles), ou alors drop les colonnes
- Gestion des valeurs manquantes -> drop les lignes ou remplacer par la moyenne ou la médiane
- Normalisation -> StandardScaler ou MinMaxScaler ou RobustScaler
- Note : Bien séparer les données en train et test avant de faire le preprocessing et s'assurer que les données de test ne sont pas utilisées pour le preprocessing

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

# Charger les données
data = load_data(n)

data.dropna(axis=0, inplace=True) # supprimer les lignes avec des valeurs manquantes

# Sélectionner les variables
X, y = select_variables(data)

# Gérer les colonnes textuelles
label_encoder = LabelEncoder()
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = label_encoder.fit_transform(X[col])

# Séparer les données en train et test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)

# Gérer les valeurs manquantes si je n'utilise pas le dropna
imputer = SimpleImputer(strategy='mean')  # ou 'median' pour la médiane
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Normaliser les données
normalizer = RobustScaler()
X_train = normalizer.fit_transform(X_train)
X_test = normalizer.transform(X_test)